In [1]:
from db_credentials	import dwh_db_connection_params
from database import Database
from variables import sourcepath
import os
import pandas as pd
import numpy as np

In [2]:
import roman

In [41]:
pap=pd.read_csv(os.path.join(sourcepath, 'papers_final.csv'))#[['journal', 'volume', 'issue', 'publisher', 'place']]
ref=pd.read_csv(os.path.join(sourcepath, 'unique_references.csv'))#[['journal', 'volume', 'issue', 'publisher', 'place']]

In [43]:
pap.head(2)

,article_id,citekey,journal_akronym,journal,year,authors,title,abstract,keywords,number_of_pages,volume,issue,publisher,place
0,1,(kevin)_yan_j_examining_interdependence_betwee...,JSIS,The Journal Of Strategic Information Systems,2021,"(Kevin) Yan, J.; Leidner, D.; Benbya, H.; Zou, W.",Examining interdependence between product user...,Firm-sponsored online user communities have be...,NaN,0.0,30.0,1.0,NaN,NaN
1,2,a_wilson_f_computer_support_for_strategic_orga...,JSIS,The Journal Of Strategic Information Systems,1994,"A Wilson, F.",Computer support for strategic organizational ...,While information systems continue to be promo...,"managerial actions, decision-making, computer-...",10.0,3.0,4,NaN,NaN


In [44]:
ref.head(2)

,citekey,authors,year,title,journal,volume,issue,pages,source_type,publisher,place,editor,monograph_title,note
0,autio_e_information_exposure_opportunity_evalu...,"Autio, E.; Dahlander, L.; Frederiksen, L.",2013,"Information Exposure, Opportunity Evaluation, ...",Acad. Manag. J,56,5,1348-1371,article,NaN,NaN,NaN,NaN,NaN
1,barrett_m_creating_value_in_online_communities...,"Barrett, M.; Oborn, E.; Orlikowski, W.",2016,Creating Value in Online Communities: The Soci...,Inform. Syst. Res,27,4,704-723,article,NaN,NaN,NaN,NaN,NaN


Option 1: in paper transformation alle funktionen implemntieren, die author-, keyword- und journal transformation auch haben und hier noch mal alles mit den article_ids ausführen um die foreign keys in der db nsuchen zu können (lookup)

Option 2: author-, keyword- und journal transformation sind class attributes von paper transformation, und beinhalten jeweils noch eine Funktion, welche den Link zu paper_id bzw citekey herstellt und ausgibt.

Option 3: einen anderen algorithmus für primary keys nutzen, der auf basis eines strings immer die gleiche nummer zurückgibt (hash value, e.g. SHA1), dann ist kein lookup nötig. Vorteil: Independent / parallel processing, child table kann vor parent table geladen werden. Kombiniert mit 1 oder 2. 

DEPENDENCY INJECTION!?
https://roelantvos.com/blog/using-a-natural-business-key-the-end-of-hash-keys/

In [9]:
print(ref.shape)
print(pap.shape)

(192003, 5)
(7479, 5)


In [5]:
all.dropna(axis=0, how='all', inplace=True)
all.shape

(174318, 5)

In [7]:
#all.fillna({'journal': 'MISSING', 'volume':0, 'issue': 0, 'publisher': 'MISSING', 'place': 'MISSING'}, inplace=True)